# Optimizing molecular structure


The initial structure can be provided manually, or as constructed as through the SMILES notation (see [previous](wf_build) section). Here we will do the latter to illustrate a full optimization workflow.

To minimize computational cost, we here perform the final quantum chemical structure optimization at a minimal level of theory (HF with the STO-3G basis set). For practical calculations, DFT or MP2 should be used.

In [1]:
import py3Dmol as p3d
from rdkit import Chem
from rdkit.Chem import AllChem
import veloxchem as vlx
import py3Dmol as p3d
import numpy as np
import geometric
import sys

ostream = vlx.OutputStream(sys.stdout)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.


# Structure of methanol

As the example, we here seek the equilibroum structure of methanol.

## Initial structure

We obtain the initial structure from RDkit, using the SMILES string (`CO`).

In [2]:
def smilestoxyz(smiles, opt = True):
    mol_bare = Chem.MolFromSmiles(smiles)
    mol_full = Chem.AddHs(mol_bare)
    AllChem.EmbedMolecule(mol_full)
    if opt: AllChem.UFFOptimizeMolecule(mol_full)
    return Chem.MolToXYZBlock(mol_full)

methanol_uff = smilestoxyz('CO')

## xTB optimization

Next, set up the xTB driver and perform the structure optimization.

In [3]:
methanol = vlx.Molecule.from_xyz_string(methanol_uff)
xtb_drv = vlx.XtbDriver()
method_settings = {'xtb':'gfn2'}
xtb_drv.set_method(method_settings['xtb'].lower())
xtb_drv.compute(methanol, ostream)
xtb_grad_drv = vlx.XtbGradientDriver(xtb_drv)
xtb_opt_drv = vlx.OptimizationDriver(xtb_grad_drv)
xtb_opt = xtb_opt_drv.compute(methanol)

                                                                                                                          
                                                        XTB Driver                                                        
                                                                                                                          

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                  12          :
          :  # atomic orbitals                  12          :
          :  # shells                            8          :
          :  # electrons                        14          :
          :  max. iterations                   280          :
          :  Hamiltonian                  GFN2-xTB          :
          :  restarted?                       true          :
          

* Info * Computing energy and gradient...                                                                                 

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                  12          :
          :  # atomic orbitals                  12          :
          :  # shells                            8          :
          :  # electrons                        14          :
          :  max. iterations                   280          :
          :  Hamiltonian                  GFN2-xTB          :
          :  restarted?                       true          :
          :  GBSA solvation                  false          :
          :  PC potential                    false          :
          :  electronic temp.          300.0000000     K    :
          :  accuracy                    1.0000000          :
        

* Info * Computing energy and gradient...                                                                                 

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                  12          :
          :  # atomic orbitals                  12          :
          :  # shells                            8          :
          :  # electrons                        14          :
          :  max. iterations                   280          :
          :  Hamiltonian                  GFN2-xTB          :
          :  restarted?                       true          :
          :  GBSA solvation                  false          :
          :  PC potential                    false          :
          :  electronic temp.          300.0000000     K    :
          :  accuracy                    1.0000000          :
        

* Info * Saving file: veloxchem_opt_2023-01-12T12.48.23_optim.xyz                                                         
* Info * Saving file: veloxchem_opt_2023-01-12T12.48.23.log                                                               
                                                                                                                          
* Info * Geometry optimization completed.                                                                                 
                                                                                                                          
                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                

Convert the results to xyz-coordinates.

In [4]:
def toxyz(molecule):
    """
    Returns the molecular geometry in xyz format.
    """
    from veloxchem.veloxchemlib import ChemicalElement
    from veloxchem.veloxchemlib import bohr_in_angstroms

    elem_ids = molecule.elem_ids_to_numpy()

    xs = molecule.x_to_numpy() * bohr_in_angstroms()
    ys = molecule.y_to_numpy() * bohr_in_angstroms()
    zs = molecule.z_to_numpy() * bohr_in_angstroms()

    txt = "" 
    txt +="%d\n\n" % molecule.number_of_atoms()
    for elem_id, x, y, z in zip(elem_ids, xs, ys, zs):
        elem = ChemicalElement()
        elem.set_atom_type(elem_id)
        txt += "%6s %22.12f %22.12f %22.12f\n" % (elem.get_name(),x,y,z)
        
    return txt

methanol_xtb = toxyz(xtb_opt)

## HF optimization

Set up the HF driver and performing the final optimization, using the xTB results as the initial guess.

```python
molecule = vlx.Molecule.from_xyz_string(methanol_xtb)
basis = vlx.MolecularBasis.read(molecule, 'STO-3G')
scf_drv = vlx.ScfRestrictedDriver()
scf_results = scf_drv.compute(molecule, basis)
grad_drv = vlx.ScfGradientDriver(scf_drv)
opt_drv = vlx.OptimizationDriver(grad_drv)
opt_molecule = opt_drv.compute(molecule, basis)
```

```python
methanol_hf = toxyz(opt_molecule)
print(f"Time elapsed: {t_end-t_start:.3f} s")
```

In [5]:
methanol_hf = '6\n\n     C        -0.355338295061         0.015832253296         0.022623869829\n     O         0.957130783496         0.544496756944        -0.204029530782\n     H        -0.479565645795        -0.368664416163         1.040410925550\n     H        -0.607609510888        -0.787365989746        -0.677656188432\n     H        -1.069087493005         0.828785331979        -0.122302379295\n     H         1.554469486523        -0.233087881063        -0.059049636847\n'

## Comparison

The three structures can be compared by visualizing the results using py3Dmol:

In [6]:
viewer = p3d.view(viewergrid=(1, 3), width=800, height=300, linked=False)
viewer.addModel(methanol_uff, 'xyz', viewer=(0, 0))
viewer.addModel(methanol_xtb, 'xyz', viewer=(0, 1))
viewer.addModel(methanol_hf, 'xyz', viewer=(0, 2))
viewer.setViewStyle({"style": "outline", "color": "black", "width": 0.1})
viewer.setStyle({"stick": {}})
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol